Hello, here is juste a simple scrapper of kenpom 2021 data in python. I also added the TeamID.

In [ ]:
# Imports
!pip install bs4
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
import requests


#--------------------------------------------------
# Scrapper

base_urls=[
          'https://kenpom.com/index.php?y=2021'
         ]

def scrap_archive(url,year):
    """
    Imports raw data from a kenpom archive into a dataframe
    """
    
    page = requests.get(url)
    soup = BeautifulSoup(page.text)
    table_full = soup.find_all('table', {'id': 'ratings-table'})

    thead = table_full[0].find_all('thead')
    table = table_full[0]
    
    for weird in thead:
        table = str(table).replace(str(weird), '')

    df = pd.read_html(table)[0]
    df['year'] = year
    
    return df
    
    
#------------------------------------------------------
# scraping
def scraping(df,year):
    
    for url in base_urls:
    
        print(f'Scrapping: {url}')
        archive=scrap_archive(url,year)
        
        df = pd.concat( (df, archive), axis=0) 
        year+=1
    
    df.columns = ['Rank', 'Team', 'Conference', 'W-L', 'Pyth', 
             'AdjustO', 'AdjustO Rank', 'AdjustD', 'AdjustD Rank',
             'AdjustT', 'AdjustT Rank', 'Luck', 'Luck Rank', 
             'SOS Pyth', 'SOS Pyth Rank', 'SOS OppO', 'SOS OppO Rank',
             'SOS OppD', 'SOS OppD Rank', 'NCSOS Pyth', 'NCSOS Pyth Rank', 'Year']
    
    df=df[[ 'Year', 'Team', 'AdjustO', 'AdjustD', 'Luck']]
    df.columns=[ 'Season', 'TeamName', 'adj_o', 'adj_d', 'luck']

    df.TeamName=df.TeamName.apply(lambda x: re.sub('\d', '', x).strip()).replace('.','')
             
        
    return df



#------------------------------------------------------
# Scrap data for 2021

df=None

year=2021

df=scraping(df,year)

df.TeamName=df.TeamName.apply(lambda x: x.replace('-',' '))
df.TeamName=df.TeamName.apply(lambda x: x.lower())
df.TeamName=df.TeamName.replace('mississippi valley st.','mississippi valley state')
df.TeamName=df.TeamName.replace('texas a&m corpus chris','texas a&m corpus christi')
df.TeamName=df.TeamName.replace('dixie st.','dixie st')
df.TeamName=df.TeamName.replace('st. francis pa','st francis pa')
df.TeamName=df.TeamName.replace('ut rio grande valley','texas rio grande valley')
df.TeamName=df.TeamName.replace('southeast missouri st.','southeast missouri state')
df.TeamName=df.TeamName.replace('tarleton st.','tarleton st')
df.TeamName=df.TeamName.replace('liu','liu brooklyn')
df.TeamName=df.TeamName.replace('cal st. bakersfield','cal state bakersfield')

#-------------------------------------------------------
# merge with spelling file to get the TeamID
spelling=pd.read_csv('../input/ncaam-march-mania-2021/MTeamSpellings.csv',encoding='cp1252')
spelling.columns=['TeamName','TeamID']
spelling.TeamName=spelling.TeamName.apply(lambda x: x.replace('-',' '))


df=df.merge(spelling[['TeamName','TeamID']],on='TeamName',how='left')

#-------------------------------------------------------
# Save the file
df.to_csv('kenpom_2021.csv',index=False)

In [ ]:
df.head()